In diesem Notebook erkläre ich kurz beispielhaft die Funktionalität der tichu_rustipy Bibliothek zum Laden, manipulieren und Auswerten der BrettSpielWelt Daten.
Wir beginnen mit dem Import:

In [3]:
import tichu_rustipy as tr

Das Laden des Datensatzes ist sehr simpel und erfolgt über die Angabe des Pfades der .db files:

In [4]:
db = tr.BSWSimple("../tichu_rust/bsw_filtered.db")

In [9]:
help(db.len)
print(db.len())
help(db.get_round)
print(db.get_round(0))

Help on built-in function len:

len() method of builtins.BSWSimple instance
    Get the total number of rounds

    Returns:
        int: The number of rounds in the BSW file

400881
Help on built-in function get_round:

get_round(index) method of builtins.BSWSimple instance
    Get a specific round's data

    Args:
        index (int): The index of the round to retrieve

    Returns:
        Optional[List[PyPlayerRoundHand]]: The round data for all players,
        or None if the index is out of bounds

[<builtins.PyPlayerRoundHand object at 0x00000289DC7BDB30>, <builtins.PyPlayerRoundHand object at 0x00000289DC7E5350>, <builtins.PyPlayerRoundHand object at 0x00000289DC7E52F0>, <builtins.PyPlayerRoundHand object at 0x00000289DC7E4F30>]


Jede gespielte Runde besteht aus einer Liste aus 4 `PyPlayerRoundHand` Objekten.
Dabei gehören Index 0 und 2 sowie 1 und 3 zum selben Team. Index 1 sitzt rechts von Index 0, Index 2 rechts von Index 1, usw.
Das `PyPlayerRoundHand` Objekt enthält alle Informationen die man braucht, und stellt einige Funktionen zur Verfügung:

In [10]:
help(tr.PyPlayerRoundHand)

Help on class PyPlayerRoundHand in module builtins:

class PyPlayerRoundHand(object)
 |  A Python wrapper for PlayerRoundHand representing a player's round in Tichu
 |
 |  This class provides access to a player's cards, exchanges, calls, and round statistics
 |  throughout different phases of a Tichu round.
 |
 |  Methods defined here:
 |
 |  is_double_win_team_1(self, /)
 |      Check if Team 1 achieved a double win
 |
 |      Returns:
 |          bool: True if Team 1 (players 0 and 2) finished 1-2
 |
 |  is_double_win_team_2(self, /)
 |      Check if Team 2 achieved a double win
 |
 |      Returns:
 |          bool: True if Team 2 (players 1 and 3) finished 1-2
 |
 |  left_in_exchange_card(self, /)
 |      Get the card received from the left player during exchange
 |
 |      Returns:
 |          CardIndex: The index of the card received from left
 |
 |  left_out_exchange_card(self, /)
 |      Get the card given to the left player during exchange
 |
 |      Returns:
 |          CardIn

Beispielsweise können über `.first_8`, `.first_14`, `.final_14` die Karten zu den jeweiligen  Zeitpunkten abgefragt werden:

In [18]:
player_0_round_0 = db.get_round(0)[0]
print(player_0_round_0.first_8)
print(player_0_round_0.first_14)
print(player_0_round_0.final_14)
print(type(player_0_round_0.final_14))

607480451170321
883594482411372561
885278938511704081
<class 'int'>


🤔 Das Ergebnis ist also jeweils ein 'int'. Wie stellt dieser denn die Hand da? Was sollen mir die Zahlen sagen?

Dafür müssen wir die Handdatenstruktur etwas besser verstehen.
In Tichu gibt es 56 Karten. Jeder Karte wird ein eindeutiger fester Index zwischen 0 und 63 gegeben. Jetzt ist das zugehörige Bit in dem Integer gesetzt (ein Integer ist ja gerade eine Menge an Bits), genau dann, wenn die Karte vorliegt oder eben nicht.

Wir kauen das an einem Beispiel nochmal durch. Zunächst bietet die Bibliothek schon etwas Funktionalität zum Darstellen einer Hand, damit wir es besser verstehen können:

In [19]:
from IPython.display import display, HTML

def display_colored_hand(hand):
    hand_str = tr.print_hand(hand)
    # Convert ANSI escape codes to HTML
    hand_str = (hand_str
        .replace('\x1b[31m', '<span style="color: red">')
        .replace('\x1b[32m', '<span style="color: green">')
        .replace('\x1b[33m', '<span style="color: yellow">')
        .replace('\x1b[34m', '<span style="color: dodgerblue ">')
        .replace('\x1b[0m', '</span>')
    )
    display(HTML(hand_str))
display_colored_hand(player_0_round_0.first_8)

Die ersten 8 Karten sind also die rote 2, die gelbe 5, blaue 8, grüne 8, grüne Q, blauer K, grünes A und der Phönix (Vogel).
PS: Wenn ihr keinen DarkMode anhabt, könnt ihr Gelb auch zu einer anderen Farbe ändern. Das ist original ja schwarz, was ich wegen dem DarkMode geändert habe :D

Das können wir auch in der Bit Representation sehen. Dafür brauchen wir aber erst einmal die Festlegung der Karten zu ihrem Index. Dafür gibt es folgendes CheatSheet:



Diese Markdown Zelle besser nicht ausführen :D
Cards:

| Red | Gre | Blu | Yel |
|  A  |  A  |  A  |  A  |
|  K  |  K  |  K  |  K  |
|  Q  |  Q  |  Q  |  Q  |
|  J  |  J  |  J  |  J  |
|  T  |  T  |  T  |  T  |
|  9  |  9  |  9  |  9  |
|  8  |  8  |  8  |  8  |
|  7  |  7  |  7  |  7  |
|  6  |  6  |  6  |  6  |
|  5  |  5  |  5  |  5  |
|  4  |  4  |  4  |  4  |
|  3  |  3  |  3  |  3  |
|  2  |  2  |  2  |  2  |
|  1  | Dr  | Dog | Ph  |

Map to the following Bit Number:

| Red | Gre | Blu | Yel |
| 61  | 45  | 29  | 13  |
| 60  | 44  | 28  | 12  |
| 59  | 43  | 27  | 11  |
| 58  | 42  | 26  | 10  |
| 57  | 41  | 25  |  9  |
| 56  | 40  | 24  |  8  |
| 55  | 39  | 23  |  7  |
| 54  | 38  | 22  |  6  |
| 53  | 37  | 21  |  5  |
| 52  | 36  | 20  |  4  |
| 51  | 35  | 19  |  3  |
| 50  | 34  | 18  |  2  |
| 49  | 33  | 17  |  1  |
| 48  | 32  | 16  |  0  |

Schauen wir uns jetzt obige Hand noch einmal in Bit Representation an:

In [20]:
bin(player_0_round_0.first_8)

'0b10001010001000000000010000100000000000000000010001'

Zählt man nach, sieht man also, dass Bit 0, Bit 4, usw. gesetzt sind, was den obigen Karten entspricht.
Je nachdem wie fit ihr mit Bit Logik seid, ist das eine sehr natürliche Darstellung. Hier noch etwas Hilfestellung, wie man so bspw. über eine Hand iterieren kann:

In [35]:
#Fundamental bit manipulation syntax in python:
# Binary typing of values: value = 0b11 (value=3)
# Bit-Or: | 
#Example: 0b01 | 0b10 = 0b11
# Bit-And: & 
#Example: 0b11 & 0b10 = 0b10
# Bit-Xor: ^ 
#Example: 0b11 ^ 0b10 = 0b01
# Not: ~
#Example: ~0b01 = 0b1111......11101 (depends on lenght of integers)
# Bit-Shift Inwards: <<
#Example 1<<3 = 0b1000
# Bit-Shift Outwards: >>
#Example 0b1001 >> 2 = 0b0010

hand = player_0_round_0.first_8
while hand !=0:
    lsb = (hand & -hand).bit_length() - 1
    print(lsb)
    hand = hand & ~(1<<lsb)

0
4
23
28
39
43
45
49


Die Nummerierung der Karten erfolgt übrigens mit System. Setzt man die Konstanten

GELB=0
BLAU=16
GRÜN=32
ROT=48

und 

SPEZIAL = 0
ZWEI=1
DREI=2

usw.
Dann ist 
BLAU+ZWEI der Index der blauen Zwei, usw.

In [46]:
def build_hand(*card_indices):
    res = 0
    for x in card_indices:
        res |= 1<<x
    return res
    
YELLOW=0
BLUE=16
GREEN=32
RED=48
SPECIAL_CARD=0
TWO=1
THREE=2
ACE=13
display_colored_hand(build_hand(BLUE+TWO))
display_colored_hand(build_hand(BLUE+TWO, ACE+RED, GREEN+THREE))
#Huch. Spezialkarten sind ... spezial:
display_colored_hand(build_hand(SPECIAL_CARD+YELLOW)) 
display_colored_hand(build_hand(SPECIAL_CARD+BLUE)) 
display_colored_hand(build_hand(SPECIAL_CARD+GREEN)) 
display_colored_hand(build_hand(SPECIAL_CARD+RED)) 

Die oben definierten Kartenindizes werden auch von einigen Funktionen zurückgegeben. Interessiert man sich bspw. für die Karten, die dem Mitspieler geschpuft wurde:

In [33]:
print(player_0_round_0.partner_out_exchange_card())
display_colored_hand(build_hand(player_0_round_0.partner_out_exchange_card()))

43


Das war es soweit an einführender Erklärung. PyPlayerRoundHand bietet noch mehr Funktionen, schaut euch dafür einfach die help Funktion (siehe Oben) nochmal genauer an. Hier noch ein paar weiterführende Beispiele:

In [45]:
#Determine number of aces in hand
aces_hand = build_hand(YELLOW+ACE, BLUE+ACE, GREEN+ACE, RED+ACE)
num_aces = (player_0_round_0.final_14 & aces_hand).bit_count()
display_colored_hand(player_0_round_0.final_14)
print(num_aces)

#Determine if player has a street of length 6 without the phoenix
FOUR, FIVE, SIX, SEVEN, EIGHT, NINE, TEN, JACK, QUEEN, KING = 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
MAHJONG = 48
MASK_YELLOW_NORMAL_CARDS = build_hand(
    TWO + YELLOW,
    THREE + YELLOW,
    FOUR + YELLOW,
    FIVE + YELLOW,
    SIX + YELLOW,
    SEVEN + YELLOW,
    EIGHT + YELLOW,
    NINE + YELLOW,
    TEN + YELLOW,
    JACK + YELLOW,
    QUEEN + YELLOW,
    KING + YELLOW,
    ACE + YELLOW
)
def has_street_length_6(hand):
    hand_in_yellow = (hand >> BLUE) | (hand >> GREEN) | (hand >> RED) | hand
    prepared_hand = (hand_in_yellow & MASK_YELLOW_NORMAL_CARDS) | ((hand >> MAHJONG) & 0b1)
    return prepared_hand & (prepared_hand >> 1) & (prepared_hand >> 2) & (prepared_hand >> 3) & (prepared_hand >> 4)& (prepared_hand >> 5) != 0

print("Has Street of length 6: ", has_street_length_6(player_0_round_0.final_14))
display_colored_hand(player_0_round_0.first_14)
print("Has Street of length 6: ", has_street_length_6(player_0_round_0.first_14))

1
Has Street of length 6:  True


Has Street of length 6:  False
